# MadryLab UROP Starter Project

Code for robust optimization algorithm as discussed in https://arxiv.org/abs/1706.06083. Written in PyTorch by Vivek Bhupatiraju.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import time

## Implementing ResNet-18 for CIFAR

Uses code from: https://github.com/kuangliu/pytorch-cifar

In [2]:
class BasicBlock(nn.Module):
  expansion = 1

  def __init__(self, in_planes, planes, stride=1):
    super(BasicBlock, self).__init__()
    self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
    self.bn1 = nn.BatchNorm2d(planes)
    self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
    self.bn2 = nn.BatchNorm2d(planes)

    self.shortcut = nn.Sequential()
    if stride != 1 or in_planes != self.expansion*planes:
        self.shortcut = nn.Sequential(
            nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
            nn.BatchNorm2d(self.expansion*planes)
        )
    
  def forward(self, x):
    out = F.relu(self.bn1(self.conv1(x)))
    out = self.bn2(self.conv2(out))
    out += self.shortcut(x)
    out = F.relu(out)
    return out

class Bottleneck(nn.Module):
  expansion = 4

  def __init__(self, in_planes, planes, stride=1):
      super(Bottleneck, self).__init__()
      self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
      self.bn1 = nn.BatchNorm2d(planes)
      self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
      self.bn2 = nn.BatchNorm2d(planes)
      self.conv3 = nn.Conv2d(planes, self.expansion*planes, kernel_size=1, bias=False)
      self.bn3 = nn.BatchNorm2d(self.expansion*planes)

      self.shortcut = nn.Sequential()
      if stride != 1 or in_planes != self.expansion*planes:
          self.shortcut = nn.Sequential(
              nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
              nn.BatchNorm2d(self.expansion*planes)
          )

  def forward(self, x):
      out = F.relu(self.bn1(self.conv1(x)))
      out = F.relu(self.bn2(self.conv2(out)))
      out = self.bn3(self.conv3(out))
      out += self.shortcut(x)
      out = F.relu(out)
      return out

class ResNet(nn.Module):
  def __init__(self, block, num_blocks, num_classes=10):
      super(ResNet, self).__init__()
      self.in_planes = 64

      self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
      self.bn1 = nn.BatchNorm2d(64)
      self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
      self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
      self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
      self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
      self.linear = nn.Linear(512*block.expansion, num_classes)

  def _make_layer(self, block, planes, num_blocks, stride):
      strides = [stride] + [1]*(num_blocks-1)
      layers = []
      for stride in strides:
          layers.append(block(self.in_planes, planes, stride))
          self.in_planes = planes * block.expansion
      return nn.Sequential(*layers)

  def forward(self, x):
      out = F.relu(self.bn1(self.conv1(x)))
      out = self.layer1(out)
      out = self.layer2(out)
      out = self.layer3(out)
      out = self.layer4(out)
      out = F.avg_pool2d(out, 4)
      out = out.view(out.size(0), -1)
      out = self.linear(out)
      return out

def ResNet18():
  return ResNet(BasicBlock, [2,2,2,2])

In [3]:
# test
net = ResNet18()
y = net(torch.randn(1,3,32,32))
print(y.size())

torch.Size([1, 10])


## Robust Training of ResNet-18 with CIFAR

### Setting Up Data

In [4]:
import torchvision
import torchvision.transforms as transforms

# prepare train and test splitss

print('==> Preparing data..')
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

==> Preparing data..
Files already downloaded and verified
Files already downloaded and verified


### RobustResNet18

In [ ]:
class RobustResNet18():
    def __init__(self):
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.net = ResNet18()
        self.net = self.net.to(self.device)
        if self.device == 'cuda':
            self.net = torch.nn.DataParallel(self.net)
            cudnn.benchmark = True
         
        self.criterion = nn.CrossEntropyLoss()
        self.optimizer = optim.SGD(self.net.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)
        
        self.best_acc = 0.0
        
    def pgd_attack(self, inputs, targets):
        self.net.eval()
        epsilon = 8.0
        num_steps = 7
        step_size = 2.0
        
        perturb = torch.FloatTensor(inputs.shape).uniform_(-epsilon, epsilon)
        perturb = perturb.to(self.device)
        
        x = inputs + perturb
        x = torch.clamp(x, 0, 255)
        x = x.to(self.device)
        
        # PGD
        for i in range(num_steps):
            x.requires_grad = True
            outputs = self.net(x)
            
            self.net.zero_grad()
            loss = self.criterion(outputs, targets)
            loss.backward()
                        
            x = torch.add(x, step_size * torch.sign(x.grad[0]))
            x = torch.max(torch.min(x, inputs+epsilon), inputs-epsilon)
            x = torch.clamp(x, 0, 255).detach()
            
        return x
        
    def robust_train(self, epoch):
        print('\nEpoch: %d' % epoch)
        start = time.time()
        train_loss = 0
        correct = 0
        total = 0
        
        for batch_idx, (inputs, targets) in enumerate(trainloader):
            inputs, targets = inputs.to(self.device), targets.to(self.device)
                        
            x = self.pgd_attack(inputs, targets) # attack inputs
            self.net.train()

            self.optimizer.zero_grad()
            outputs = self.net(x)
            loss = self.criterion(outputs, targets)
            loss.backward()
            self.optimizer.step()

            train_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
        
        acc = 100.*correct/total
        end = time.time()
        
        print()
        print("Time Elapsed for Training: " + str(end - start))
        print("Training Loss: " + str(train_loss))
        print("Accuracy: " + str(acc))
        print()

    def robust_test(self, epoch):
        start = time.time()
        
        self.net.eval()
        test_loss = 0
        correct = 0
        total = 0
        
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(self.device), targets.to(self.device)
            
            x = self.pgd_attack(inputs, targets) # attack inputs
            
            outputs = self.net(inputs)
            loss = self.criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
            
        acc = 100.*correct/total
        self.best_acc = max(self.best_acc, acc)
        end = time.time()

        print("Time Elapsed for Testing: " + str(end - start))
        print("Testing Loss: " + str(test_loss))
        print("Accuracy: " + str(acc))
        print("Best Accuracy: " + str(self.best_acc))
        print()
    
    def run(self):
        for epoch in range(100):
            self.robust_train(epoch)
            self.robust_test(epoch)

net = RobustResNet18()
net.run()

### Robust Net without Classes

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
net = ResNet18()
net = net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)

best_acc = 0.0
train_losses = []
train_accs = []
test_losses = []
test_accs = []

def pgd_attack(inputs, targets):    
    net.eval()
    epsilon = 8.0
    num_steps = 20
    step_size = 2.0

    perturb = torch.FloatTensor(inputs.shape).uniform_(-epsilon, epsilon)
    perturb = perturb.to(device)

    x = inputs + perturb
    x = torch.clamp(x, 0, 255)
    x = x.to(device)
    
    # PGD
    for i in range(num_steps):   
        x.requires_grad = True
        outputs = net(x)
        
        net.zero_grad()
        loss = criterion(outputs, targets)
        loss.backward()
                
        x = torch.add(x, step_size * torch.sign(x.grad[0]))
        x = torch.max(torch.min(x, inputs+epsilon), inputs-epsilon)
        x = torch.clamp(x, 0, 255).detach()
        
    return x
        
def robust_train(epoch):
    print('\nEpoch: %d' % epoch)
    start = time.time()
    train_loss = 0
    correct = 0
    total = 0

    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        
        x = pgd_attack(inputs, targets)
        net.train()
        
        optimizer.zero_grad()
        outputs = net(x)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
            
    acc = 100.*correct/total
    end = time.time()

    print()
    print("Time Elapsed for Training: " + str(end - start))
    print("Training Loss: " + str(train_loss))
    print("Accuracy: " + str(acc))
    print()
    
    train_losses.append(train_loss)
    train_accs.append(acc)
    
def robust_test(epoch):
    global best_acc
    start = time.time()

    net.eval()
    test_loss = 0
    correct = 0
    total = 0

    for batch_idx, (inputs, targets) in enumerate(testloader):
        inputs, targets = inputs.to(device), targets.to(device)
        
        x = pgd_attack(inputs, targets)

        outputs = net(x)
        loss = criterion(outputs, targets)

        test_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        
    acc = 100.*correct/total
    best_acc = max(best_acc, acc)
    end = time.time()

    print("Time Elapsed for Testing: " + str(end - start))
    print("Testing Loss: " + str(test_loss))
    print("Accuracy: " + str(acc))
    print("Best Accuracy: " + str(best_acc))
    print()
    
    test_losses.append(test_loss)
    test_accs.append(acc)

# training
for epoch in range(100):
    robust_train(epoch)
    robust_test(epoch)


Epoch: 0

Time Elapsed for Training: 978.7786481380463
Training Loss: 879.2984795570374
Accuracy: 16.69

Time Elapsed for Testing: 243.38013792037964
Testing Loss: 1586.755474805832
Accuracy: 10.19
Best Accuracy: 10.19


Epoch: 1

Time Elapsed for Training: 945.4443252086639
Training Loss: 849.8282160758972
Accuracy: 18.948

Time Elapsed for Testing: 241.55784010887146
Testing Loss: 3615.3089735507965
Accuracy: 10.04
Best Accuracy: 10.19


Epoch: 2

Time Elapsed for Training: 951.5105540752411
Training Loss: 839.1685342788696
Accuracy: 20.554

Time Elapsed for Testing: 241.72223615646362
Testing Loss: 1646.2651734352112
Accuracy: 9.8
Best Accuracy: 10.19


Epoch: 3

Time Elapsed for Training: 958.4364037513733
Training Loss: 829.5473036766052
Accuracy: 21.762

Time Elapsed for Testing: 243.15978360176086
Testing Loss: 1536.2897880077362
Accuracy: 9.53
Best Accuracy: 10.19


Epoch: 4

Time Elapsed for Training: 952.4777073860168
Training Loss: 816.0189071893692
Accuracy: 23.106

Time E